# Facial Emotion Recognition using Deep Learning

The goal of the project is to develop a deep learning model that can accurately classify facial expressions into one of six categories: Angry, Disgust, Fear, Happy, Sad, and Surprise. We will use convolutional neural networks (CNNs) to train our model, as they have been shown to be effective in image classification tasks. Our approach involves training a CNN model from scratch using the FER-2013 dataset.

## Data
The FER-2013 dataset consists of 48x48 pixel grayscale images of faces that have been centered and that occupy a similar amount of space in each image. 

## Importing libraries

In [4]:
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt
from torch.optim import Adam
from torch.utils.data import DataLoader
import deeplake
from tqdm import tqdm

In [ ]:
ds = deeplake.open('hub://activeloop/fer2013-train')

Exception: The API for Deep Lake 4.0 has changed significantly, including the `load` method being replaced by `open`. To continue using Deep Lake 3.x, use `pip install "deeplake<4"`. For information on migrating your code, see https://docs.deeplake.ai/latest/details/v3_conversion/